In [1]:
# # # # Web scraping: Africa in International Math Olympiads # # # # 

In [2]:
# Importing packages

import requests,math
import numpy as np
from bs4 import BeautifulSoup

In [3]:
# # Data about performance of countries at the IMO

URL_imo = 'https://www.imo-official.org/results.aspx'
page_imo = requests.get(URL_imo)
imo = BeautifulSoup(page_imo.content, 'html.parser')
results_imo = imo.find(id='main')

countries_imo = results_imo.find_all('tr')
countries_imo.pop(0)
countries_imo.pop(0)

year = [results_imo.find_all('tr')[0]('th')[i]('a')[0]['title'][4:] for i in range(1,len(results_imo.find_all('tr')[0]('th'))-1)]
code = [0 for i in range(len(countries_imo))] # country codes
name = [0 for i in range(len(countries_imo))] # country names
data = [[float('nan') for y in year] for i in code] # country performance at IMO

for i in range(len(countries_imo)):
    code[i] = countries_imo[i].a.text
    name[i] = countries_imo[i].a['title']
    
for c in range(len(code)):
    for y in range(len(year)):
        if not (countries_imo[c].find_all('td')[y+1].a is None): # making sure <a> exists
            if ("team_r.aspx?" in countries_imo[c].find_all('td')[y+1].a["href"]): # if the country c participated the year y
                data[c][y] = int(countries_imo[c].find_all('td')[y+1].a.text)

In [4]:
# # African countries

URL_africa = 'https://allafrica.com/misc/sitemap/countries.html' # list of African countries
page_africa = requests.get(URL_africa)
africa = BeautifulSoup(page_africa.content, 'html.parser')
results_africa = africa.find(class_='category-grid')
countries_africa = [results_africa.find_all('a')[c].text.replace(u'\xa0', u' ') for c in range(1,len(results_africa.find_all('a')))]

In [5]:
# # Need to identify African countries (linking the two data bases)

#african_participant = [0 for c in name] # value 0 if African country never participated to IMO
african_participant = []

i=0
for c in range(len(name)):
    if name[c] in countries_africa:
        #african_participant[c] = 1 # value 1 if African country  participated to at least one IMO edition
        african_participant.append(c) # their position in IMO countries list ('name')
    i=i+1

africa_performance = [data[a] for a in african_participant] # performance of African countries

In [6]:
# # Some statistics

len(african_participant) # 19 African countries at the IMO

np.nanmin(africa_performance,axis=1) # best performance for each African country
name[african_participant[np.argmin(np.nanmin(africa_performance,axis=1))]],np.nanmin(africa_performance),year[np.nanargmin(data[african_participant[np.argmin(np.nanmin(africa_performance,axis=1))]])]
# Morocco achieved the best performance for an African country, 18, in 1982

np.nanmax(africa_performance),sum(sum(africa_performance[a]==np.nanmax(africa_performance) for a in range(len(africa_performance)-1))) # worst performance accross all African countries:110. Achieved 3 times

(110.0, 3)

In [6]:
# Relative performance: rank divided by number of participants

data_standardized = [[float('nan') for y in year] for i in code]

for i in range(len(code)):
    for y in range(len(year)):
        if not(math.isnan(data[i][y])):
            data_standardized[i][y] = np.divide(data[i][y],np.nanmax(data,axis=0)[y])
            
africa_performance_standardized = [data_standardized[a] for a in african_participant] # performance of African countries

In [9]:
np.nanmin(africa_performance_standardized,axis=1) # best relative performance for each African country
name[african_participant[np.argmin(np.nanmin(africa_performance_standardized,axis=1))]],np.nanmin(africa_performance_standardized),year[np.nanargmin(data[african_participant[np.argmin(np.nanmin(africa_performance_standardized,axis=1))]])]
# Morocco achieved the best relative performance for an African country in 1982

('Morocco', 0.3287671232876712, '1986')

In [ ]:
# Plot: for each African country, evolution of its performance at IMO

In [ ]:
# Map: for each African country, number of its participations at IMO

# Help 1: https://ramiro.org/notebook/basemap-choropleth/ # Not working properly

# Help 2: https://medium.com/using-specialist-business-databases/creating-a-choropleth-map-using-geopandas-and-financial-data-c76419258746

In [ ]:
# Needed to install geopandas through Anaconda Prompt, by running the code "conda install -c conda-forge geopandas"

In [7]:
import pandas as pd #used to read in the revenue file 
import matplotlib.pyplot as plt #for plotting
#to read in shape file and provides high #level interface with #matplotlib library for making maps
import geopandas as gpd

# Available data sets
# gpd.datasets.available

#import world shape map externally, no need to have a saved file
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# view first 5 rows
# world.head()

#inspect count
# world.info()

In [67]:
# merge both data sets using country code/iso_a3 as unique identifiers

score = pd.DataFrame({'abs_rank':np.nanmin(data,axis=1),'rel_rank':np.nanmin(data_standardized,axis=1)},index=code) # consider working with DataFrames from the beginning.

for_plotting = world.merge(score, left_on = 'iso_a3', right_index=True)
#check the occurrence
#for_plotting.info()